<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/goldSilverRatio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
from kiteconnect import KiteConnect
import pandas as pd
import numpy as np
import logging
import os
import time
import datetime as dt
api_key = "1ll4p8ziqlk8hh7g"
api_secret = "x59r5u1vd25annfpgts22wv8ykfueirl"
kite = KiteConnect(api_key=api_key)
print(kite.login_url()) #use this url to manually login and authorize yourself

https://kite.zerodha.com/connect/login?api_key=1ll4p8ziqlk8hh7g&v=3


In [ ]:
#generate trading session
request_token = "SjadY458xC3aRziUBIEgM6Imakf0XKf3" #Extract request token from the redirect url obtained after you authorize yourself by loggin in
data = kite.generate_session(request_token, api_secret=api_secret)
#create kite trading object
kite.set_access_token(data["access_token"])

#get dump of all NSE instruments
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv("NSE_Instruments.csv",index=False)

# get dump of all MCX instruments
instrument_dump_mcx = kite.instruments("MCX")
instrument_df_mcx = pd.DataFrame(instrument_dump_mcx)
instrument_df_mcx.to_csv("MCX_Instruments.csv",index=False)


def instrumentLookup(instrument_df_nse, instrument_df_mcx, symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        # Try to find the instrument in NSE
        token_nse = instrument_df_nse[instrument_df_nse.tradingsymbol == symbol].instrument_token.values
        if token_nse.size > 0:
            return token_nse[0]
        # If not found in NSE, try to find in MCX
        token_mcx = instrument_df_mcx[instrument_df_mcx.tradingsymbol == symbol].instrument_token.values
        if token_mcx.size > 0:
            return token_mcx[0]
        return -1 # Return -1 if not found in either
    except Exception as e:
        print(f"Error looking up instrument {symbol}: {e}")
        return -1

def fetchOHLC(ticker,interval,duration):
  """extracts historcial data and outputs in the form of dataframe"""
  instrument = instrumentLookup(instrument_df, instrument_df_mcx, ticker)
  if instrument != -1:
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration),dt.date.today(),interval))
    # data1 = pd.DataFrame(kite.historical_data(instrument,dt.date.today(),dt.date.today(),interval))
    # print(data1.head())
    data.set_index("date",inplace=True)
    return data
  else:
    print(f"Instrument {ticker} not found.")
    return pd.DataFrame() # Return empty DataFrame if instrument not found

In [ ]:
import http.client
import json

# Adjust these number to suit your needs
buy_silver_ratio_min = 66
buy_gold_ratio_max = 40

payload = ''
headers = {
  'x-access-token': 'goldapi-dwpk2uki92tuzg-io',
  'Content-Type': 'application/json'
}
conn = http.client.HTTPSConnection("www.goldapi.io")

# Get current silver prpayloadice.
conn.request("GET", "/api/XAG/USD", payload, headers)
res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))
silver_price = data['price']

# Get current gold price.
conn.request("GET", "/api/XAU/USD", payload, headers)
res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))
gold_price = data['price']

gold_silver_ratio = round(gold_price/silver_price, 2)
print(f"Today's gold to silver ratio is: {gold_silver_ratio}")

if gold_silver_ratio >= buy_silver_ratio_min:
    print("Buy Silver!")
elif buy_gold_ratio_max <= gold_silver_ratio > buy_silver_ratio_min:
    print("Hang tight.  Don't buy either.")
elif gold_silver_ratio < buy_gold_ratio_max:
    print("Buy Gold!")

Today's gold to silver ratio is: 86.34
Buy Silver!
